### The Promise Container

In [63]:
from pymonad.promise import Promise, _Promise

In [64]:
Promise.insert(1)

In [48]:
await Promise.insert(1)

1

### Map Method

In [4]:
import asyncio

async def wait_and_then_add_one(x: int) -> int:
    print(f"before waiting {x} sec...")
    await asyncio.sleep(x)
    print(f"after waiting {x} sec...")
    return x+1

In [5]:
def add_one(x: int) -> int:
    return x + 1

In [65]:
p = Promise.insert(1).map(wait_and_then_add_one).map(add_one)

In [50]:
p

In [66]:
await Promise.insert(1).map(wait_and_then_add_one).map(wait_and_then_add_one)

before waiting 1 sec...
after waiting 1 sec...
before waiting 2 sec...
after waiting 2 sec...


3

### Bind method

In [20]:
def generate_new_promise(x: int) -> _Promise[int]:
    print("generating a new promise")
    return Promise.insert(x).map(wait_and_then_add_one)

In [67]:
p = Promise.insert(1).bind(generate_new_promise)

In [68]:
await p

generating a new promise
before waiting 1 sec...
after waiting 1 sec...


2

### Then method 

In [69]:
await Promise.insert(1).then(generate_new_promise).then(wait_and_then_add_one)

generating a new promise
before waiting 1 sec...
after waiting 1 sec...
before waiting 2 sec...
after waiting 2 sec...


3

### Catching Exception

In [70]:
def raise_exception_when_too_large(upper_bound: int):
    def wrapper(value: int):
        if value > upper_bound:
            raise ValueError(f"Too large. Value must be lower than {upper_bound}")
        return value
    return wrapper

In [71]:
max_limit = 2

In [72]:
p = Promise.insert(1)\
        .then(generate_new_promise)\
        .then(wait_and_then_add_one)\
        .then(raise_exception_when_too_large(2))

In [73]:
await p

generating a new promise
before waiting 1 sec...
after waiting 1 sec...
before waiting 2 sec...
after waiting 2 sec...


ValueError: Too large. Value must be lower than 2

In [40]:
def resetting_to_upper_bound(upper_bound: int):
    def wrapper(e: Exception):
        print(f"Raised Exception: {e} \n Falling back to {upper_bound}")
        return upper_bound
    return wrapper

In [74]:
safe_p = p = Promise.insert(1)\
        .then(generate_new_promise)\
        .then(raise_exception_when_too_large(1))\
        .then(wait_and_then_add_one)\
        .catch(resetting_to_upper_bound(max_limit))

In [75]:
await safe_p

generating a new promise
before waiting 1 sec...
after waiting 1 sec...
Raised Exception: Too large. Value must be lower than 1 
 Falling back to 2


2